In [1]:
import requests
import lxml.html

In [2]:
url="https://declarator.org/office/453"
root="https://declarator.org"

In [3]:
# This requires the installation of Splash
r=requests.get("http://localhost:8050/render.html",params={"url":url})

In [4]:
html=lxml.html.fromstring(r.content)

In [5]:
new_dict={}
for element in html.xpath('//tr'):
    year=element.xpath('./td[position()=1]/text()')[0]
    element_2=element.xpath('./td[position()=2]//ul[@class="list-inline activetext"]/li/a')[0]
    if element_2.xpath('./text()')[0].strip()=="Anti-corruption declaration":
        anti_corrupt_url=root+element_2.xpath("./@href")[0]
        r2=requests.get("http://localhost:8050/render.html",params={'url':anti_corrupt_url})
        html2=lxml.html.fromstring(r2.content)
        #Get people
        person=html2.xpath('//tr[@class="section_summary " or @class="section_summary even  "]/td[@class="person"]')
        #Get details
        element_final=html2.xpath('//tr[@class="section_detailed" or @class="section_detailed even "]/td[@colspan="3" or @colspan="4"]')
        assert len(element_final)==len(person)
        url_list=[]
        for element_person in person:
            url_person=element_person.xpath('./div[@class="person_name activetext"]/a/@href')
            url_list.append(root+url_person[0])
        new_dict[year]=list(zip(person,element_final,url_list))

In [6]:
def extract_name_element(selector_element):
    title=selector_element.xpath('./div[2]/i/text()')[0]
    family_name=selector_element.xpath('./div[1]/a/div[@class="family_name"]/h4/text()')[0]
    name=selector_element.xpath('./div[1]/a/h4/text()')[0]
    return [("family_name",family_name),("name",name),("title",title)]

In [7]:
def extract_asset_details(selector_element):
    return selector_element.xpath('.//text()')

In [8]:
import re

def clean_text(list_of_words):
    clean_list=[re.sub(r"\n","",word).strip() for word in list_of_words]
    clean_list=[re.sub(r" m"," sqm",word).strip() for word in clean_list]
    clean_list=[item for item in clean_list if len(item)>0 and item!="2"]
    return clean_list

def clean_text_details(list_of_words):
    clean_list=clean_text(list_of_words)
    new_list=[]
    i=0
    while i <=len(clean_list)-1:
        if i==len(clean_list)-1:
            new_list.append(clean_list[i])
            i+=1
        elif clean_list[i+1][0]=="(" or clean_list[i+1][0]==",":
            new_list.append(clean_list[i]+" "+clean_list[i+1])
            i+=2
        else:
            new_list.append(clean_list[i])
            i+=1
    return new_list

def clean_text_name(list_of_words):
    clean_list=clean_text(list_of_words)
    #full_name=", ".join(clean_list)
    return clean_list

def clean_text_title(list_of_words):
    clean_list=clean_text(list_of_words)
    full_title=" ".join(clean_list)
    return full_title

In [9]:
new_dict_2={}
for key in new_dict.keys():
    new_dict_2[key]=[extract_name_element(item_1)+[("url",item_3)]+[clean_text_details(extract_asset_details(item_2))] for item_1,item_2,item_3 in new_dict[key]]

In [10]:
def create_dicts(year,family_name,name,title,url,cleaned_list):
    index_list=[]
    for item in ["Income","Real estate","Transport","Accounts"]:
        try:
            index=cleaned_list.index(item)
            index_list.append(index)
        except:
            continue
    final_output=[]
    index_list.append(len(cleaned_list))
    for i in range(len(index_list)-1):
        begin=index_list[i]+1
        end=index_list[i+1]
        span=cleaned_list[begin:end]
        for item in span:
            return_list={}
            return_list['last_name']=family_name[1]
            return_list['first_name']=name[1]
            return_list["year"]=year
            return_list["link"]=url[1]
            return_list["position"]=title[1]
            return_list["asset_type"]=cleaned_list[index_list[i]]
            return_list["asset_value"]=item
            final_output.append(return_list)  
    return final_output

In [11]:
final_dicts=[]

In [12]:
for key in new_dict_2.keys():
    for entry in new_dict_2[key]:
        entry_dict=create_dicts(key,entry[0],entry[1],entry[2],entry[3],entry[4])
        final_dicts.extend(entry_dict)
        #print(entry_dict)

In [13]:
import pandas as pd
output_df=pd.DataFrame(final_dicts)

This code is to extract the Russian names

In [18]:
def find_russian_name(x):
    root_person="http://declarator.org/person_report/"
    id_person=re.findall(r"[0-9]+",x)[0]
    full_id_url=root_person+id_person+"/"
    df_name=pd.read_excel(full_id_url)
    return df_name['ФИО'].unique()[0]

In [19]:
list_of_russian_names=[find_russian_name(link) for link in output_df.link.unique()]

In [20]:
df_russian_name=pd.DataFrame({"id":output_df.link.unique(),"russian_name":list_of_russian_names})

In [21]:
output_df_2=output_df.merge(df_russian_name,how="left",right_on="id",left_on="link").drop(['id'],axis=1)

In [31]:
output_df_2

,last_name,first_name,year,link,position,asset_type,asset_value,russian_name
0,Abramchenko,Viktoriia Valerievna,2020,https://declarator.org/person/22071/,Заместитель председателя правительства РФ,Income,"18 718 910,85 руб.",Абрамченко Виктория Валериевна
1,Abramchenko,Viktoriia Valerievna,2020,https://declarator.org/person/22071/,Заместитель председателя правительства РФ,Income,"Супруг(а): 3 587 691,40 руб.",Абрамченко Виктория Валериевна
2,Abramchenko,Viktoriia Valerievna,2020,https://declarator.org/person/22071/,Заместитель председателя правительства РФ,Real estate,"Land plot, 1400 sqm",Абрамченко Виктория Валериевна
3,Abramchenko,Viktoriia Valerievna,2020,https://declarator.org/person/22071/,Заместитель председателя правительства РФ,Real estate,"Residential house, 359 sqm",Абрамченко Виктория Валериевна
4,Abramchenko,Viktoriia Valerievna,2020,https://declarator.org/person/22071/,Заместитель председателя правительства РФ,Real estate,"Apartment, 85.2 sqm",Абрамченко Виктория Валериевна
...,...,...,...,...,...,...,...,...
6504,Shchegolev,Igor Olegovich,2008,https://declarator.org/person/1382/,Minister of communications and mass communicat...,Income,"4 714 800,00 руб.",Щеголев Игорь Олегович
6505,Shchegolev,Igor Olegovich,2008,https://declarator.org/person/1382/,Minister of communications and mass communicat...,Income,"Супруг(а): 310 800,00 руб.",Щеголев Игорь Олегович
6506,Shchegolev,Igor Olegovich,2008,https://declarator.org/person/1382/,Minister of communications and mass communicat...,Real estate,"Apartment, 75.7 sqm (lease )",Щеголев Игорь Олегович
6507,Shchegolev,Igor Olegovich,2008,https://declarator.org/person/1382/,Minister of communications and mass communicat...,Real estate,"Супруг(а): Apartment, 139 sqm",Щеголев Игорь Олегович


In [32]:
output_df_2.to_excel("Russia_federal_government_anti_corruption.xlsx",index=False)

In [33]:
output_df_2.to_csv("Russia_federal_government_anti_corruption.csv",index=False)

32